In [2]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [1]:
from fastai.imports import *
from fastai.structured import *

from pandas.api.types import is_categorical_dtype
from pandas.api.types import is_string_dtype, is_object_dtype
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from IPython.display import display
import feather
from rfpimp import *

import seaborn as sns
import matplotlib.pyplot as plt
from pdpbox import pdp
from plotnine import *

from sklearn.metrics import mean_squared_error, mean_absolute_error
from scipy.cluster import hierarchy as hc
from sklearn import metrics
from treeinterpreter import treeinterpreter as ti


/home/heatmap/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/heatmap/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
np.random.seed(8)

# 1. Loading data

In [21]:
PATH = 'data/bulldozer/'

In [22]:
df= pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=['saledate'])
df_test_raw= pd.read_csv(f'{PATH}Valid.csv', low_memory=False, parse_dates=['saledate'])

In [23]:
os.makedirs('data/bulldozer/feather', exist_ok=True)
df = df.sort_values('saledate')
n_valid = 12000  # same as Kaggle's test set size
n_train = len(df)-n_valid
df_train_raw = df[:n_train].reset_index(drop=True)
df_valid_raw = df[n_train:].reset_index(drop=True)

df_train_raw.to_feather('data/bulldozer/feather/train')
df_valid_raw.to_feather('data/bulldozer/feather/valid')
df_test_raw.to_feather('data/bulldozer/feather/test')

# 2. Cleaning data

In [24]:
def string_norm(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            df[col] = df[col].str.lower()
            df[col] = df[col].fillna(np.nan) 
            df[col] = df[col].replace('none or unspecified', np.nan)
            df[col] = df[col].replace('none', np.nan)
            df[col] = df[col].replace('', np.nan)
            
def remove_inch(df, colname):
    df[colname] = df[colname].str.extract(r'([0-9.]*)', expand=True)
    df[colname] = pd.to_numeric(df[colname])            

def clean(df):
    del df['MachineID'] # dataset has inconsistencies
    del df['SalesID']   # unique sales ID so not generalizer

    df['auctioneerID'] = df['auctioneerID'].astype(str)

    string_norm(df)

    remove_inch(df, 'Tire_Size')
    remove_inch(df, 'Undercarriage_Pad_Width')

    df.loc[df['YearMade']<1950, 'YearMade'] = np.nan
    df.loc[df.eval("saledate.dt.year < YearMade"), 'YearMade'] = \
        df['saledate'].dt.year

    df.loc[df.eval("MachineHoursCurrentMeter==0"),
           'MachineHoursCurrentMeter'] = np.nan

# 3. Feature Engineering

In [25]:
def df_order_product_size(df):
    sizes = {np.nan:0, 'mini':1, 'compact':1, 'small':2, 'medium':3,
             'large / medium':4, 'large':5}
    df['ProductSize'] = df['ProductSize'].map(sizes).values
    
def onehot(df, colname):
    ascat = df[colname].astype('category').cat.as_ordered()
    onehot = pd.get_dummies(df[colname], prefix=colname, dtype=bool)
    del df[colname]
    df = pd.concat([df, onehot], axis=1)
    return df, ascat.cat.categories

def split_fiProductClassDesc(df):
    df_split = df.fiProductClassDesc.str.split(' - ',expand=True).values
    df['fiProductClassDesc'] = df_split[:,0] 
    df['fiProductClassSpec'] = df_split[:,1] 
    pattern = r'([0-9.\+]*)(?: to ([0-9.\+]*)|\+) ([a-zA-Z ]*)'
    spec = df['fiProductClassSpec']
    df_split = spec.str.extract(pattern, expand=True).values
    df['fiProductClassSpec_lower'] = pd.to_numeric(df_split[:,0])
    df['fiProductClassSpec_upper'] = pd.to_numeric(df_split[:,1])
    df['fiProductClassSpec_units'] = df_split[:,2]
    del df['fiProductClassSpec'] 
    
def prep_date(df, date_col):
        df["saleYear"] = df[date_col].dt.year
        df["saleMonth"] = df[date_col].dt.month
        df["saleDay"] = df[date_col].dt.day
        df["saleDayofweek"] = df[date_col].dt.dayofweek
        df["saleWeekofyear"] = df[date_col].dt.weekofyear
        df["saleDayofyear"] = df[date_col].dt.dayofyear
        df["saleQuarter"] = df[date_col].dt.quarter
        df["saleIsWeekend"] = np.where((df[date_col].dt.day_name == ['Sunday', 'Saturday']),1,0)
        df["saleIsQuarterStart"] = np.where(df[date_col].dt.month.isin([1, 4, 7, 10]),1 ,0)
        df["saleIsQuarterEnd"] = np.where(df[date_col].dt.month.isin([3, 6, 9, 12]),1 ,0)
        df.drop(date_col, axis=1, inplace=True)

In [26]:

def feature_eng(X): # for later use
    prep_date(X, 'saledate')
    df_order_product_size(X)
    split_fiProductClassDesc(X)

    X, hf_cats = onehot(X, 'Hydraulics_Flow')
    # normalize categories first then one-hot encode
    X['Enclosure'] = X['Enclosure'].replace('erops w ac', 'erops ac')
    X['Enclosure'] = X['Enclosure'].replace('no rops', np.nan)
    X, enc_cats = onehot(X, 'Enclosure')
    catencoders = {'Hydraulics_Flow':hf_cats,
                   'Enclosure':enc_cats}

    return X, catencoders


# 4. Numericalize

In [27]:
def fix_missing_num (df, col):
    df[col+'_na'] = pd.isnull(df[col]) 
    df[col].fillna(df[col].median(), inplace=True)

def df_fix_missing_nums(df:pd.DataFrame) -> dict:
    medians = {}  
    for colname in df.columns:
        if is_numeric_dtype(df[colname]):
            medians[colname] = df[colname].median(skipna=True)
            fix_missing_num(df, colname)
    return medians

def df_string_to_cat(df:pd.DataFrame) -> dict:
    catencoders = {}
    for colname in df.columns:
        if is_string_dtype(df[colname]) or is_object_dtype(df[colname]):
            df[colname] = df[colname].astype('category').cat.as_ordered()
            catencoders[colname] = df[colname].cat.categories
    return catencoders

def df_cat_to_catcode(df):
    for col in df.columns:
        if is_categorical_dtype(df[col]):
            df[col] = df[col].cat.codes + 1


def numericalize(X, catencoders):
    medians = df_fix_missing_nums(X)            
    e = df_string_to_cat(X)
    catencoders.update(e)
    df_cat_to_catcode(X)    
    return medians

## 4.1 Prepare training set


In [28]:
df = pd.read_feather("data/bulldozer/feather/train")
df = df.iloc[-100_000:] 
X_train, y_train = df.drop('SalePrice', axis=1), df['SalePrice']

In [29]:
y_train = np.log(y_train)
clean(X_train)
X_train, catencoders = feature_eng(X_train)
medians = numericalize(X_train, catencoders)

## 4.2 Prepare the validation set

Due to problems like data leakage and to ensure consistency we will fill NaNs in the validation/test sets with the medians from our training set.

In [31]:
def df_fix_missing_test_nums(df_test, medians):
    for colname in medians:
        df_test[colname+'_na'] = pd.isnull(df_test[colname])
        df_test[colname].fillna(medians[colname], inplace=True)

In the next step we must ensure that our validation/test set consists of the exact same categories as our training set. Categories which are in the validation/test set but not in the training set will be encoded as NaNs and later as zeros.

In [32]:
def df_apply_cats(df_test:pd.DataFrame, catencoders:dict):
    for colname,encoder in catencoders.items():
        # encode with categories from training set
        df_test[colname] = \
            pd.Categorical(df_test[colname],
                           categories=encoder, ordered=True)

The same goes for one-hot encoded columns. If a column in the validation data has more levels than the same column in the training data we end up with a validation set with more columns than the training set. To ensure this does not happen categories which are only in the validation/test set will be coded as NaN.

In [33]:
def onehot_apply_cats(df_test, colname, catencoders_oh):
    df_test[colname] = \
        pd.Categorical(df_test[colname],
                       categories=catencoders[colname],
                       ordered=True)
    onehot = pd.get_dummies(df_test[colname], prefix=colname, dtype=bool)
    del df_test[colname]
    df_test = pd.concat([df_test, onehot], axis=1)
    del catencoders[colname] # simplify df_apply_cats()
    return df_test

Adjusted feature engeneering function for the validation/test set.

In [34]:
def feature_eng_test(df_test, catencoders):
    prep_date(df_test, 'saledate')
    df_order_product_size(df_test)
    split_fiProductClassDesc(df_test)

    df_test = onehot_apply_cats(df_test, 'Hydraulics_Flow', catencoders)
    df_test['Enclosure'] = df_test['Enclosure'].replace('erops w ac', 'erops ac')
    df_test['Enclosure'] = df_test['Enclosure'].replace('no rops', np.nan)
    df_test = onehot_apply_cats(df_test, 'Enclosure', catencoders)

    return df_test

And numericalize everything as a last step.

In [35]:
def numericalize_test(df_test:pd.DataFrame, medians:dict, catencoders:dict):
    df_apply_cats(df_test, catencoders)
    df_fix_missing_test_nums(df_test, medians)
    df_cat_to_catcode(df_test)

Now we just have to apply the functions above to our validation set.

In [36]:
df_valid = pd.read_feather("data/bulldozer/feather/valid")
X_valid, y_valid = df_valid.drop('SalePrice', axis=1), df_valid['SalePrice']

y_valid = np.log(y_valid)
clean(X_valid)
X_valid = feature_eng_test(X_valid, catencoders)
numericalize_test(X_valid, medians, catencoders)

## 4.2 Sanity Check

To ensure our train and validation sets line up cocorrectly we can run a quick sanity check.

In [37]:
def sanity_check(df):
    for col in df.columns:
        if is_string_dtype(df[col]) or is_object_dtype(df[col]):
            print(f"Col {col} is still a string")
        if df[col].isnull().any():
            print(f"Col {col} still has missing values")

def check_types(df1,df2):
    if df1.shape[1] != df2.shape[1]:
        print(f"Num columns differs: {df1.shape[1]} != {df2.shape[1]}")
    cols1 = set(df1.columns)
    cols2 = set(df2.columns)
    if cols1 != cols2:
        print(f"Column names differ:")
        if len(cols1-cols2)>0:
            print(f"\tIn df1 not df2: {cols1-cols2}")
        if len(cols2-cols1)>0:
            print(f"\tIn df2 not df1: {cols2-cols1}")
    for col in cols1.intersection(cols2): # check those in common
        if df1[col].dtype != df2[col].dtype:
            print(f"Col {col} dtypes differ {df1[col].dtype} != {df2[col].dtype}")

In [38]:
sanity_check(X_train)
sanity_check(X_valid)
check_types(X_train, X_valid)

Finally we just have to make sure that the columns in training and validations sets have the same order.

In [39]:
X_valid = X_valid.reindex(columns=X_train.columns)

Define metrics

In [40]:
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

def print_score(m):
    res = ["Train_RMSLE:", rmse(m.predict(X_train), y_train), "Test_RMSLE:",rmse(m.predict(X_valid), y_valid),
               "R²_Train:", m.score(X_train, y_train), "R²_Test:", m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(['OOB: ', m.oob_score_])
    print(res)

In [41]:
m = RandomForestRegressor(n_estimators=40, min_samples_leaf=3, max_features=0.5,
                          n_jobs=-1, oob_score=True)
m.fit(X_train, y_train)
print_score(m)

['Train_RMSLE:', 0.12404969772314627, 'Test_RMSLE:', 0.23797961327470485, 'R²_Train:', 0.9691045334838347, 'R²_Test:', 0.8916507983514057, ['OOB: ', 0.9150222850602242]]


# 5. Tuning

## 5.1 Subsetting the data for faster training

For tuning we will work with a smaller subset of our data to speed up training time. With new data we first have to prepare train and validation set in the same manner as above.

In [42]:
df = pd.read_feather("data/bulldozer/feather/train")
df = df.query('saledate.dt.year>=2007').copy()
X_train, y_train = df.drop('SalePrice', axis=1), df['SalePrice']

In [43]:
X_train.shape

(172411, 52)

In [44]:
y_train = np.log(y_train)
clean(X_train)
X_train, catencoders = feature_eng(X_train)
medians = numericalize(X_train, catencoders)

df_valid = pd.read_feather("data/bulldozer/feather/valid")
X_valid, y_valid = df_valid.drop('SalePrice', axis=1), df_valid['SalePrice']

y_valid = np.log(y_valid)
clean(X_valid)
X_valid = feature_eng_test(X_valid, catencoders)
df_apply_cats(X_valid, catencoders)
df_fix_missing_test_nums(X_valid, medians)
df_cat_to_catcode(X_valid)

In [45]:
sanity_check(X_train)
sanity_check(X_valid)
check_types(X_train, X_valid)

## 5.2 Train, measure and report

In [46]:
def test_valid(X_train, y_train, X_valid, y_valid, n_estimators=200,
               max_features='auto', min_samples_leaf=1):
    X_valid = X_valid.reindex(columns=X_train.columns)
    m = RandomForestRegressor(n_estimators=n_estimators,
                               n_jobs=-1,
                               oob_score=True,
                               max_features=max_features, 
                               min_samples_leaf=min_samples_leaf)
    m.fit(X_train, y_train)
    y_pred = m.predict(X_valid)
    mae_valid = mean_absolute_error(np.exp(y_valid), np.exp(y_pred))
    rmsle_valid = np.sqrt( mean_squared_error(y_valid, y_pred))
    r2_score_valid = m.score(X_valid, y_valid)
    print(f"OOB R^2 {m.oob_score_:.5f}")
    print(f"Validation R^2 {r2_score_valid:.5f}, RMSLE {rmsle_valid:.5f}, MAE ${mae_valid:.0f}")
    return m, r2_score_valid, rmsle_valid, mae_valid
    

In [47]:
ntrees = 200
minleaf = 1
for maxf in np.arange(.1,.7,.1):
    print(f"n_estimators={ntrees}, max_features={maxf:.1f}, min_samples_leaf={minleaf}")
    test_valid(X_train, y_train, X_valid, y_valid,
               max_features=maxf, min_samples_leaf=minleaf)

n_estimators=200, max_features=0.1, min_samples_leaf=1
OOB R^2 0.90850
Validation R^2 0.87832, RMSLE 0.25220, MAE $6436
n_estimators=200, max_features=0.2, min_samples_leaf=1
OOB R^2 0.92004
Validation R^2 0.89127, RMSLE 0.23840, MAE $5915
n_estimators=200, max_features=0.3, min_samples_leaf=1
OOB R^2 0.92247
Validation R^2 0.89409, RMSLE 0.23529, MAE $5784
n_estimators=200, max_features=0.4, min_samples_leaf=1
OOB R^2 0.92304
Validation R^2 0.89541, RMSLE 0.23382, MAE $5701
n_estimators=200, max_features=0.5, min_samples_leaf=1
OOB R^2 0.92283
Validation R^2 0.89535, RMSLE 0.23389, MAE $5678
n_estimators=200, max_features=0.6, min_samples_leaf=1
OOB R^2 0.92228
Validation R^2 0.89513, RMSLE 0.23412, MAE $5674


The model with max_features= 0.5 performs best. So we will keep it steady while optimizing min_samples_leaf.

In [48]:
maxf = .5
for minleaf in range(2,7):
   print(f"n_estimators={ntrees}, max_features={maxf}, min_samples_leaf={minleaf}")
   test_valid(X_train, y_train, X_valid, y_valid,
              max_features=maxf, min_samples_leaf=minleaf)

n_estimators=200, max_features=0.5, min_samples_leaf=2
OOB R^2 0.92128
Validation R^2 0.89546, RMSLE 0.23376, MAE $5683
n_estimators=200, max_features=0.5, min_samples_leaf=3
OOB R^2 0.91891
Validation R^2 0.89517, RMSLE 0.23409, MAE $5708
n_estimators=200, max_features=0.5, min_samples_leaf=4
OOB R^2 0.91675
Validation R^2 0.89483, RMSLE 0.23447, MAE $5729
n_estimators=200, max_features=0.5, min_samples_leaf=5
OOB R^2 0.91512
Validation R^2 0.89453, RMSLE 0.23480, MAE $5741
n_estimators=200, max_features=0.5, min_samples_leaf=6
OOB R^2 0.91325
Validation R^2 0.89360, RMSLE 0.23583, MAE $5774


Here the best performance is achieved with min_samples_leaf = 2 or 3 depending on the run(According to the law of large numbers the fluctuation could be reduced by increasing the number of trees, the more trees the more stable are the results.). We can now train the model with the full data set again and see how we perform.

In [49]:
df = pd.read_feather("data/bulldozer/feather/train")
X_train, y_train = df.drop('SalePrice', axis=1), df['SalePrice']

In [50]:
y_train = np.log(y_train)
clean(X_train)
X_train, catencoders = feature_eng(X_train)
medians = numericalize(X_train, catencoders)

df_valid = pd.read_feather("data/bulldozer/feather/valid")
X_valid, y_valid = df_valid.drop('SalePrice', axis=1), df_valid['SalePrice']

y_valid = np.log(y_valid)
clean(X_valid)
X_valid = feature_eng_test(X_valid, catencoders)
df_apply_cats(X_valid, catencoders)
df_fix_missing_test_nums(X_valid, medians)
df_cat_to_catcode(X_valid)

In [51]:
test_valid(X_train, y_train, X_valid, y_valid,
              max_features=0.5, min_samples_leaf=2)

OOB R^2 0.91714
Validation R^2 0.89722, RMSLE 0.23178, MAE $5663


(RandomForestRegressor(max_features=0.5, min_samples_leaf=2, n_estimators=200,
                       n_jobs=-1, oob_score=True),
 0.8972222387265705,
 0.23178027478410357,
 5662.904534915931)

## 5.3 Feature Importance

While random forests are fairly robust and don't care that much on how many features they are trained we might improve performance by removing unimportant features.

In [52]:
def select_features(X_train, y_train, X_valid, y_valid, drop=0.10):
   min_rmsle = 99999
   X_valid = X_valid.reindex(columns=X_train.columns)
   m, _, rmsle, _ = test_valid(X_train, y_train, X_valid, y_valid,
                                max_features=.4, min_samples_leaf=2)
   I = importances(m, X_valid, y_valid)
   features = list(I.index)
   keep = best_features = features
   n = int(.9/drop) # how many iterations? get to 90%
   for i in range(1,n+1):
       X2_train = X_train[keep]
       X_valid2 = X_valid[keep]
       print(f"\nNum features = {len(keep)}")
       m2, _, rmsle, _ = test_valid(X2_train, y_train, X_valid2, y_valid,
                                     max_features=.4, min_samples_leaf=2)
       if rmsle < min_rmsle:
           min_rmsle = rmsle
           best_features = keep
       I2 = importances(m2, X_valid2, y_valid) # recompute since collinear
       features = list(I2.index)
       keep = features[0:int(len(features)*(1-drop))]

   return min_rmsle, best_features

In [53]:
min_rmsle, best_features = \
    select_features(X_train, y_train, X_valid, y_valid, drop=0.05)
print(f"{len(best_features)} features is best:")
print(best_features)

OOB R^2 0.91665
Validation R^2 0.89674, RMSLE 0.23233, MAE $5701

Num features = 89
OOB R^2 0.91648
Validation R^2 0.89733, RMSLE 0.23166, MAE $5685

Num features = 84
OOB R^2 0.91665
Validation R^2 0.89999, RMSLE 0.22864, MAE $5587

Num features = 79
OOB R^2 0.91654
Validation R^2 0.89960, RMSLE 0.22908, MAE $5591

Num features = 75
OOB R^2 0.91650
Validation R^2 0.89984, RMSLE 0.22881, MAE $5583

Num features = 71
OOB R^2 0.91645
Validation R^2 0.89989, RMSLE 0.22875, MAE $5572

Num features = 67
OOB R^2 0.91241
Validation R^2 0.90194, RMSLE 0.22639, MAE $5457

Num features = 63
OOB R^2 0.91147
Validation R^2 0.90255, RMSLE 0.22570, MAE $5443

Num features = 59
OOB R^2 0.91134
Validation R^2 0.90247, RMSLE 0.22578, MAE $5444

Num features = 56
OOB R^2 0.84960
Validation R^2 0.89540, RMSLE 0.23383, MAE $5705

Num features = 53
OOB R^2 0.84983
Validation R^2 0.89649, RMSLE 0.23260, MAE $5688

Num features = 50
OOB R^2 0.84988
Validation R^2 0.89565, RMSLE 0.23354, MAE $5692

Num featur

In [54]:
X_train = X_train[best_features]
X_valid = X_valid[best_features]
rf, r2_score_bestf, rmsle_bestf, mae_bestf = \
    test_valid(X_train, y_train, X_valid, y_valid,
               max_features=.5, min_samples_leaf=2)

OOB R^2 0.91161
Validation R^2 0.90185, RMSLE 0.22651, MAE $5446


With hyperparameter tuning and feature selection we went from a RMSLE of .240 down to .225.

## 5.4 Adjust for inflation

In [55]:
y_valid_pred = rf.predict(X_valid)
underprediction = np.mean(y_valid-y_valid_pred)
dollars = np.mean(np.exp(y_valid)-np.exp(y_valid_pred))
print(f"Model underpredicts by ${dollars:.0f}, {underprediction:.5f} log(dollars)")

Model underpredicts by $1657, 0.01699 log(dollars)


In [56]:
y_valid_pred = rf.predict(X_valid) + underprediction
mae_best = mean_absolute_error(np.exp(y_valid), np.exp(y_valid_pred))
rmsle_best = np.sqrt( mean_squared_error(y_valid, y_valid_pred) )
r2_score_best = r2_score(y_valid, y_valid_pred)
print(f"Adjusted-model validation R^2 {r2_score_best:.5f}, RMSLE {rmsle_best:.5f}, MAE {mae_best:.0f}")

Adjusted-model validation R^2 0.90240, RMSLE 0.22587, MAE 5391


# 6. Final Model + Test

In [57]:
df= pd.read_csv(f'{PATH}Train.csv', low_memory=False, parse_dates=['saledate'])
df = df.query('saledate.dt.year>=2007').copy()
X_train, y_train = df.drop('SalePrice', axis=1), df['SalePrice']
y_train = np.log(y_train)
clean(X_train)
X_train, catencoders = feature_eng(X_train)
medians = numericalize(X_train, catencoders)
X_train = X_train[best_features]

In [58]:
df_test = pd.read_feather("data/bulldozer/feather/test")
X_test = df_test
tmp = pd.read_csv(f'{PATH}ValidSolution.csv')
y_test = np.log(tmp['SalePrice'])
clean(X_test)
X_test = feature_eng_test(X_test, catencoders)
df_apply_cats(X_test, catencoders)
df_fix_missing_test_nums(X_test, medians)
df_cat_to_catcode(X_test)
X_test = X_test[best_features]

In [59]:
sanity_check(X_train)
sanity_check(X_valid)
check_types(X_train, X_valid)

In [60]:
X_test.shape, y_test.shape


((11573, 63), (11573,))

In [61]:
m, r2_score_test, rmsle_test, mae_test = \
    test_valid(X_train, y_train + underprediction,
               X_test, y_test,
               max_features=.5, min_samples_leaf=2)

OOB R^2 0.91478
Validation R^2 0.89657, RMSLE 0.23620, MAE $5831


# 6. Tree interpretation